In [1]:
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import seaborn as sns
import helpers
import dataset
import test_lstm
import tensorflow as tf
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2022-12-02 00:17:21.525568: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries

# MU3D_path = '/Users/frank/Downloads/dataSets/MU3D-dataset/processed/' # 320 entries

In [3]:
X, Y = dataset.path_preprocessing(truth_trial_path, lie_trial_path)

In [4]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [14]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=False)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))

  model.add(Dropout(drop))
  
  model.add(Dense(1, activation='sigmoid')) #softmax

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                6272      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1536/1536 [==============================] - 8s 4ms/step - loss: 0.4494 - accuracy: 0.7806 - val_loss: 0.3509 - val_accuracy: 0.8241
Epoch 2/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.3220 - accuracy: 0.8410 - val_loss: 0.2540 - val_accuracy: 0.8790
Epoch 3/10
1536/1536 [======

In [6]:
print(MODEL.predict(xTrain))

1536/1536 [==============================] - 3s 1ms/step
[[ 0.78196037]
 [ 1.208529  ]
 [ 0.28925908]
 ...
 [-0.170787  ]
 [ 1.6159518 ]
 [ 0.777282  ]]


In [21]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Deceptive/processed/'

test_lstm.perdictSingleVideo(lie_test_path, MODEL)

40/40 [==============================] - 0s 1ms/step
[[0.99498516]
 [0.99498516]
 [0.99498516]
 ...
 [0.99498516]
 [0.99498516]
 [0.99498516]]
